# Abbaspour - 610398147 - Classifying MNIST (without Encoding Images)
The problem is import raw image for classification task. This can be done by means of Convolutional Neural Networks and VGG16 pretrained model.
First some important methods from well-known libraries are imported

In [ ]:
from tensorflow.keras import Input, Sequential
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.layers import BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.models import Model
# from tensorflow.keras.optimizers import Adam, Nadam, SGD
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array
from tensorflow.keras.utils import to_categorical
from matplotlib.pyplot import figure, imshow, ion, legend, plot, savefig, show, subplots, title, xlabel, ylabel
from numpy import argmax, asarray, dstack, mean, sqrt, unique
from os import environ
from pandas import read_csv
from seaborn import histplot
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import StratifiedKFold
from warnings import filterwarnings
filterwarnings("ignore")
environ["KMP_DUPLICATE_LIB_OK"], environ["TF_CPP_MIN_LOG_LEVEL"], environ["TF_ENABLE_ONEDNN_OPTS"] = "TRUE", "3", "0"

# A Callback for Plotting
A Keras callback class is implemented for saving test loss and accuracy epoch by epoch for plotting metrics behaviours when epoch gets bigger.

In [ ]:
# A class for plotting evaluation of model's performance on test set epoch by epoch
class TestCallback(Callback):
    def __init__(self):
        self.loss, self.acc = [], []

    def on_epoch_end(self, epoch, logs = {}):
        Loss, accuracy = self.model.evaluate(X_test, caty_test, verbose = 0)
        self.loss.append(Loss)
        self.acc.append(accuracy)
        # print(f"Testing loss: {Loss}, Accuracy: {accuracy}")

# Loading Dataset & Defining Variables
Train set and test set are read as data frames. Accoring to histogram plot located in 'Evaluations' folder, whole dataset is roughly balanced and dataset doesn't seem to be resampled.
As a preprocessing, values are divided by 255 as they're in range (0, 255) as a normalization.
The reason of this action is to avoid the possibility of exploding gradients due to the high range the pixels and improve the convergence speed.
Number of 3 x 3 filters in the first convolutional layer equals 6, and fully connected layer is implemented with two hidden layer includes 8 and 4 neurons, respectively. Small values elected due to Trainset is also halved and due to complexity of VGG16 model whose training process divegres on my current system!
Labelset are one-hot encoded (categorical form) due to categorical crossentropy loss function.
A 5-fold structure is also implemeted.

In [ ]:
# Loading & normalizing the dataset
trainSet, testSet, inputLayer, numberFilters, filterSize, ReLU, Softmax, poolSize, Strides, lossFunction, Accuracy, Loss, validationAccuracy, validationLoss, Epochs, Title, Alpha, localPopulation, Numbers, numberIterations, numberGenerations, Fitness, Parameters, Optimizer, Legends, Location, XLabel, YLabel, Blue, Green, Red, figureSize, DPI, train_batch_size, test_batch_size, validationSplit, OFF, Axis, Tuner = read_csv("mnist_train.csv"), read_csv("mnist_test.csv"), Input(shape = (48, 48, 3)), [2, 4, 6, 8, 10], (3, 3), "relu", "softmax", (2, 2), 2, "categorical_crossentropy", "accuracy", "loss", "val_accuracy", "val_loss", 15, "Convolutional Neural Network Evaluation", 0.6, 1, 10, 1, 1, [], [], "adam", ["Train Loss", "Validation Loss", "Test Loss"], "upper right", "Epochs", "Loss", "blue", "green", "red", (13, 5), 1200, 1000, 100, 0.2, 0, 1, {}
df, Length, rangeEpochs, Metric, Beta = trainSet._append(testSet, ignore_index = True), int(sqrt(trainSet.shape[1])), range(Epochs), [Accuracy], 1 - Alpha
X_train, X_test, y_train, y_test = trainSet.drop(["label"], axis = 1).astype("float32") / 255., testSet.drop(["label"], axis = 1).astype("float32") / 255., trainSet["label"].astype("int"), testSet["label"].astype("int")
dfX_train, localBound, number_train_samples, number_test_samples, numberClasses,  = X_train, X_train.shape[1], len(X_train) // 2, len(X_test), len(unique(y_test)), 

# Halving train set due to complexity of VGG16 model!
subsequentLayers, X_train, y_train = [BatchNormalization(), MaxPooling2D(pool_size = poolSize, strides = Strides), Flatten(), Dense(8, activation = ReLU), Dense(4, activation = ReLU), Dense(numberClasses, activation = Softmax)], X_train.iloc[:number_train_samples], y_train.iloc[:number_train_samples]
X_train, X_test, caty_train, caty_test = X_train.to_numpy().reshape(-1, 28, 28, 1), X_test.to_numpy().reshape(-1, 28, 28, 1), to_categorical(y_train, numberClasses), to_categorical(y_test, numberClasses)
KFold, inputShape = StratifiedKFold(n_splits = 5, shuffle = True).split(X_train, y_train), X_train.shape[1:]
priorLayers, TCB = [Input(shape = inputShape), Conv2D(6, kernel_size = filterSize, activation = ReLU), BatchNormalization(), MaxPooling2D(pool_size = poolSize, strides = Strides)], TestCallback()
print(f"There're {number_train_samples} samples of {Length} X {Length} images as train samples and {number_test_samples} samples as test samples.")

# Plotting dataset
Images, histogram and pie chart of dataset are plotted.

In [ ]:
# Plot samples
for Number in range(Numbers):
    fig = figure
    imshow(arrX_train[Number], cmap = "gray")
    savefig(dpi = 1200)
    show()

# Plotting raw data records
Figure, ax = subplots()
ax.pie(df["label"].value_counts(), labels = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"], autopct = '%1.1f%%', shadow = True, startangle = 90)
ax.axis("equal")
savefig(dpi = 1200)
show()
histplot(df["label"])
savefig("MNIST Histogram", dpi = 1200)
show()

# Training Convolutional Neural Network Classifier
CNN model is constructed according to the recipe of homework especially by means of 5-fold cross validation approach for choosing a good value as the number of 3 x 3 filters for the second convolutional layer.
Size of filters and maxpoolers are 3 x 3 and 2 x 2, respectively due to their well-known performance while doesn't have many trainable parameters. 

In [ ]:
print("\nTuning classifier...")
for Filters in numberFilters:
    Accuracies = []
    for Train, Validation in KFold:
        # Create model
        Classifier = Sequential(priorLayers + [Conv2D(Filters, kernel_size = filterSize, activation = ReLU)] + subsequentLayers)

        Classifier.compile(loss = lossFunction,  metrics = Metric, optimizer = Optimizer)

        # Fit the model
        Classifier.fit(X_train[Train], caty_train[Train], batch_size = 128, epochs = 3, verbose = OFF)

        # Evaluate the model
        Accuracies.append(Classifier.evaluate(X_train[Validation], caty_train[Validation], batch_size = 64, verbose = OFF)[1] * 100)
    Tuner[Filters] = mean(Accuracies)
    # Store the mean score and the filter number
    
print("\n\nTraining the classifier...\n")
# Defining the model
CNNClassifier = Sequential(priorLayers + [Conv2D(max(Tuner, key = Tuner.get), kernel_size = filterSize, activation = ReLU)] + subsequentLayers)

# Compiling the model
CNNClassifier.compile(loss = lossFunction, metrics = Metric, optimizer = Optimizer)
print(CNNClassifier.summary())
CNNHistory = CNNClassifier.fit(X_train, caty_train, batch_size = train_batch_size, callbacks = [TCB], epochs = Epochs, validation_split = validationSplit, verbose = OFF)

# Saving model
CNNClassifier.save("CNN Classifier.keras")

# CNN-based Classifier Evaluation Plot
Final loss value and accuracy of model's perfomance on train set, validaiton set and test set are plotted.
Confusion matrix is also plotted.

In [ ]:
# Plotting the training history (losses)
figure(figsize = figureSize)
ion()
plot(CNNHistory.history[Loss], Blue)
plot(CNNHistory.history[validationLoss], Green)
plot(rangeEpochs, TCB.loss, Red)
title(Title)
ylabel(YLabel)
xlabel(XLabel)
legend(Legends, loc = Location)
savefig(Title, dpi = DPI)
show()

# Plot the training history (accuracies)
figure(figsize = figureSize)
ion()
plot(CNNHistory.history[Accuracy], Blue)
plot(CNNHistory.history[validationAccuracy], Green)
plot(rangeEpochs, TCB.acc, Red)
title(Title)
ylabel(Accuracy)
xlabel(XLabel)
legend(Legends, loc = Location)
savefig(Title, dpi = DPI)
show()

Title = "Convolutional Neural Network Confusion Matrix"
print(f"\nEvaluating classifier...\n\nModel's score: {CNNClassifier.evaluate(X_test, caty_test, batch_size = test_batch_size, verbose = OFF)}")
ConfusionMatrixDisplay(confusion_matrix = confusion_matrix(y_test, argmax(CNNClassifier.predict(X_test, batch_size = test_batch_size, verbose = OFF), axis = Axis))).plot()
title(Title)
savefig(Title, dpi = DPI)
show()

# Datasets New Preprocessing
Train set and test set are reshaped to get prepared for being used for VGG16 pretrained model.

In [ ]:
# Data preprocessing for being input for VGG16 model
X_train, X_test, TCB, Title = dstack([X_train] * 3).reshape(-1, 28, 28, 3), dstack([X_test] * 3).reshape (-1, 28, 28, 3), TestCallback(), "VGG16 Neural Network Evaluation"
X_train, X_test = asarray([img_to_array(array_to_img(Image, scale = False).resize((48, 48))) for Image in X_train]), asarray([img_to_array(array_to_img(Image, scale = False).resize((48, 48))) for Image in X_test])

# VGG16 Classifier Construction & Training
VGG16 model is pretrained so that all its weights (except the weights of its fully connected layer (without top)) are imported and become freezed avoid of being trained again. (weights.trainable = False) 
Now fully connected layers constructed above are reused for constructing customized VGG16-based model to get ready for being trained.

In [ ]:
# Loading pretrained VGG16 model
VGG16Model = VGG16(input_tensor = inputLayer, include_top = False, weights = None)
VGG16Model.load_weights("VGG16 Pretrained Models/VGG16 Weights (without Top).h5")
VGG16Layers = VGG16Model.layers

# Freezing all the layers of VGG16 model to avoid them from being trained again!
for VGG16Layer in VGG16Layers:
    VGG16Layer.trainable = False

print(VGG16Model.summary())

# Creating model with input and output layer by means of VGG16 pretrained model
VGG16Classifier = Model(inputs = inputLayer, outputs = Dense(numberClasses, activation = Softmax)(Dense(4, activation = ReLU)(Dense(8, activation = ReLU)(Flatten()(VGG16Model.output)))))

print(VGG16Classifier.summary())

VGG16Classifier.compile(loss = lossFunction, metrics = Metric, optimizer = Optimizer)

# Train the model
VGG16History = VGG16Classifier.fit(X_train, caty_train, batch_size = train_batch_size, callbacks = [TCB], epochs = Epochs, validation_split = validationSplit, verbose = OFF)

# VGG16-based Classifier Evaluation Plot
Final loss value and accuracy of model's perfomance on train set, validaiton set and test set are plotted.
Confusion matrix is also plotted.

In [ ]:
# Plotting the training history (losses)
figure(figsize = figureSize)
ion()
plot(VGG16History.history[Loss], Blue)
plot(VGG16History.history[validationLoss], Green)
plot(rangeEpochs, TCB.loss, Red)
title(Title)
ylabel(YLabel)
xlabel(XLabel)
legend(Legends, loc = Location)
savefig(Title, dpi = DPI)
show()

# Plot the training history (accuracies)
figure(figsize = figureSize)
ion()
plot(VGG16History.history[Accuracy], Blue)
plot(VGG16History.history[validationAccuracy], Green)
plot(rangeEpochs, TCB.acc, Red)
title(Title)
ylabel(Accuracy)
xlabel(XLabel)
legend(Legends, loc = Location)
savefig(Title, dpi = DPI)
show()

Title = "VGG16 Confusion Matrix"
print(f"\nEvaluating VGG16 classifier...\n\nModel's score: {VGG16Classifier.evaluate(X_test, caty_test, batch_size = test_batch_size, verbose = OFF)}")
ConfusionMatrixDisplay(confusion_matrix = confusion_matrix(y_test, argmax(VGG16Classifier.predict(X_test, batch_size = test_batch_size, verbose = OFF), axis = Axis))).plot()
title(Title)
savefig(Title, dpi = DPI)
show()

# Results
According to plots in 'Evaluations' folder, at the end of each epoch among 15 number of epochs (epochs are small due to time and hardware limitations), both models make accuracy and loss increased, and decreased for all three train, validation and test sets which means there're no overfitting occurences so that there's no need to use drop out layers which avoid being overfitted a little.
Also CNN-based classifier did better (which concluded accuracy +80%) than VGG16 (with accuracy roughly 17%) maybe because of smallness of number of epochs which made model not be trained enough.

# Challenges
Main challenge was to construct VGG16-based classifier model while the pretrained model has a big size in bytes, and it's big enough to run out RAM memory usage which caused crahsing and giving up continuing running.
So a good batch size which does convegre both training processes as well is elected (equals 1000) and training is decided to be done in 15 epochs due to time and hardware limits.

# Appendix
# Build of VGG16 Model
The input to cov1 layer is of fixed size 224 x 224 RGB image. The image is passed through a stack of convolutional (conv.) layers, where the filters were used with a very small receptive field: 3×3 (which is the smallest size to capture the notion of left/right, up/down, center). In one of the configurations, it also utilizes 1×1 convolution filters, which can be seen as a linear transformation of the input channels (followed by non-linearity). The convolution stride is fixed to 1 pixel; the spatial padding of conv. layer input is such that the spatial resolution is preserved after convolution, i.e. the padding is 1-pixel for 3×3 conv. layers. Spatial pooling is carried out by five max-pooling layers, which follow some of the conv. layers (not all the conv. layers are followed by max-pooling). Max-pooling is performed over a 2×2 pixel window, with stride 2.
Three Fully-Connected (FC) layers follow a stack of convolutional layers (which has a different depth in different architectures): the first two have 4096 channels each, the third performs 1000-way ILSVRC classification and thus contains 1000 channels (one for each class). The final layer is the soft-max layer. The configuration of the fully connected layers is the same in all networks.
All hidden layers are equipped with the rectification (ReLU) non-linearity. It is also noted that none of the networks (except for one) contain Local Response Normalisation (LRN), such normalization does not improve the performance on the ILSVRC dataset, but leads to increased memory consumption and computation time.


# Transfer Learning
Transfer learning (TL) is a technique in machine learning (ML) in which knowledge learned from a task is re-used in order to boost performance on a related task. For example, for image classification, knowledge gained while learning to recognize cars could be applied when trying to recognize trucks. This topic is related to the psychological literature on transfer of learning, although practical ties between the two fields are limited.
Reusing/transferring information from previously learned tasks to new tasks has the potential to significantly improve learning efficiency.
Since transfer learning makes use of training with multiple objective functions it is related to cost-sensitive machine learning and multi-objective optimization.
Transfer learning uses pre-trained models from one machine learning task or dataset to improve performance and generalizability on a related task or dataset.
Transfer learning is a machine learning technique in which knowledge gained through one task or dataset is used to improve model performance on another related task and/or different dataset. In other words, transfer learning uses what has been learned in one setting to improve generalization in another setting Transfer learning has many applications, from solving regression problems in data science to training deep learning models. Indeed, it is particularly appealing for the latter given the large amount of data needed to create deep neural networks.
Traditional learning processes build a new model for each new task, based on the available labeled data. This is because traditional machine learning algorithms assume training and test data come from the same feature space, and so if the data distribution changes, or the trained model is applied to a new dataset, users must retrain a newer model from scratch, even if attempting a similar task as the first model (e.g. sentiment analysis classifier of movie reviews versus song reviews). Transfer learning algorithms, however, takes already-trained models or networks as a starting point. It then applies that model’s knowledge gained in an initial source task or data (e.g. classifying movie reviews) towards a new, yet related, target task or data (e.g. classifying song reviews)